In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [ ]:
batch_size = 128
epochs = 20
lr = 0.001
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load Datasets
dataset = ClassificationDataset('/content')
testset = ClassificationDataset('/content', mode='test')
trainloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

In [ ]:
# Model, Loss, Optimizer
model = CNNModel(num_classes=100).to(device)
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

In [ ]:
# Helper Functions
def accuracy(y_pred, y):
    _, predicted = torch.max(y_pred.data, 1)
    total = y.size(0)
    correct = (predicted == y).sum().item()
    return correct/total

def train(model, dataset, optimizer, criterion, device):
    train_iter = iter(dataset)
    model.train()
    acc_vals = []
    loss_vals = []
    for i, (data, targets) in enumerate(train_iter):
        data = data.to(device=device)
        targets = targets.to(device=device)

        outputs = model.forward(data)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = accuracy(outputs, targets)
        acc_vals.append(acc)
        loss_val = loss.item()
        loss_vals.append(loss_val)

    return np.mean(acc_vals), np.mean(loss_vals)

def test(model, dataset, criterion, device):
    model.eval()
    acc_vals = []
    loss_vals = []
    with torch.no_grad():
        for data, targets in dataset:
            data = data.to(device=device)
            targets = targets.to(device=device)

            outputs = model.forward(data)
            loss = criterion(outputs, targets)

            acc = accuracy(outputs, targets)
            acc_vals.append(acc)
            loss_val = loss.item()
            loss_vals.append(loss_val)

    return np.mean(acc_vals), np.mean(loss_vals)

In [ ]:
# Training Loop
train_accs = []
test_accs = []
train_losses = []
test_losses = []

for epoch in range(epochs):
    print(epoch)
    train_acc, train_loss = train(model, trainloader, optimizer, loss, device)
    test_acc, test_loss = test(model, testloader, loss, device)
    train_accs.append(train_acc)
    test_accs.append(test_acc)
    train_losses.append(train_loss)
    test_losses.append(test_loss)

print("OVERALL TRAINING COMPLETE")
print(f"Train accuracy: {max(train_accs): .6f}")
print(f"Test accuracy: {max(test_accs): .6f}")
